In [1]:
# Librerias
import pandas as pd
import numpy as np

In [2]:
# Ruta al archivo
ruta = "../data/processed/df_limpio_violencia.csv"  

# Lectura del CSV
df = pd.read_csv(
    ruta,
    encoding="cp1252",
    low_memory=False
)

In [3]:
DF_violencia_mujer = df.copy()

In [4]:
# Categorizamos que respuestas consideramos violencia
violencia_si = [
    "de vez en cuando",
    "frecuentemente",
    "pocas veces",
    "una vez",
    "muchas veces"
]

# Evaluamos las 2 col. de la infacia en funcion de la lista de 
# violencia_si si existio episodio de violencia marcamos con 1 si no 0
DF_violencia_mujer["violencia_infancia"] = (

    DF_violencia_mujer[[
        "f2_s5_3", "f2_s5_5"
    ]]
    .isin(violencia_si)
    .any(axis = 1)
    .astype(int)
)

In [5]:
DF_violencia_mujer['violencia_pareja'] = (

    DF_violencia_mujer[[
        "f2_s7a_14_16",
        "f2_s7a_14_17",
        "f2_s7a_14_18",
        "f2_s7a_14_19",
        "f2_s7a_14_20"
    ]]
    .isin(violencia_si)
    .any(axis = 1) #Revisa por fila para cualquiera de la col. una es verdadero devuelve True
    .astype(int)
)

In [6]:
DF_violencia_mujer.head()

,f2_s5_1,f2_s5_3,f2_s5_5,f2_s7a_14_16,f2_s7a_14_17,f2_s7a_14_18,f2_s7a_14_19,f2_s7a_14_20,violencia_infancia,violencia_pareja
0,una ciudad,nunca,nunca,una vez,no ocurrió,no ocurrió,no ocurrió,no ocurrió,0,1
1,"una localidad, comunidad o recinto",nunca,nunca,pocas veces,pocas veces,pocas veces,pocas veces,pocas veces,0,1
2,"una localidad, comunidad o recinto",de vez en cuando,de vez en cuando,no ocurrió,no ocurrió,no ocurrió,no ocurrió,no ocurrió,1,0
3,"una localidad, comunidad o recinto",de vez en cuando,nunca,no ocurrió,no ocurrió,no ocurrió,no ocurrió,no ocurrió,1,0
4,una ciudad,nunca,nunca,no ocurrió,no ocurrió,no ocurrió,no ocurrió,no ocurrió,0,0


In [7]:
# Cuenta las combinaciones posibles para ambas columnas
pd.crosstab(
    DF_violencia_mujer['violencia_infancia'],
    DF_violencia_mujer['violencia_pareja']
)

violencia_pareja,0,1
violencia_infancia,,
0,5411,956
1,2830,1380


In [8]:
# Porcentaje de las cobminaciones de ambas columnas
pd.crosstab(
    DF_violencia_mujer['violencia_infancia'],
    DF_violencia_mujer['violencia_pareja'],
    normalize = True
)*100

violencia_pareja,0,1
violencia_infancia,,
0,51.158173,9.038480
1,26.756169,13.047178


In [9]:
# Porcentaje de violencia en infancia
DF_violencia_mujer['violencia_infancia'].value_counts(normalize = True).mul(100).round(2)

violencia_infancia
0    60.2
1    39.8
Name: proportion, dtype: float64

In [10]:
# Porcentaje de violencia en pareja
DF_violencia_mujer['violencia_pareja'].value_counts(normalize = True).mul(100).round(2)

violencia_pareja
0    77.91
1    22.09
Name: proportion, dtype: float64

Insight
- Casos:
    - 5411: No sufrieron violencia ni en infancia ni en pareja (0,0)
    - 2830: Sufrieron violencia en infancia pero no en pareja (1,0)
    - 1380:Sufrieron violencia tanto en infancia como en pareja(1,1)
    - 956:No sufrieron violencia en infancia pero si en pareja (0,1)

- Casos:
    - 51% : No sufrieron violencia ni en infancia ni en pareja (0,0)
    - 26,7%: Sufrieron violencia en infancia pero no en pareja (1,0)
    - 13%:Sufrieron violencia tanto en infancia como en pareja(1,1)
    - 9%:No sufrieron violencia en infancia pero si en pareja (0,1)

Para continuar nuestra investigacion nos interesa el grupo (1,1) y (0,1) Porque:
- Nos centramos en el grupo de mujeres con pareja que sufren violencia o sufrieron con su pareja
- Comparamos las mujeres que sufren violencia con su pareja y tuvieron una infancia con violencia vs mujeres que sufren violencia con su pereja pero no tuvieron violencia en su infancia

In [11]:
DF_violencia_mujer.head()

,f2_s5_1,f2_s5_3,f2_s5_5,f2_s7a_14_16,f2_s7a_14_17,f2_s7a_14_18,f2_s7a_14_19,f2_s7a_14_20,violencia_infancia,violencia_pareja
0,una ciudad,nunca,nunca,una vez,no ocurrió,no ocurrió,no ocurrió,no ocurrió,0,1
1,"una localidad, comunidad o recinto",nunca,nunca,pocas veces,pocas veces,pocas veces,pocas veces,pocas veces,0,1
2,"una localidad, comunidad o recinto",de vez en cuando,de vez en cuando,no ocurrió,no ocurrió,no ocurrió,no ocurrió,no ocurrió,1,0
3,"una localidad, comunidad o recinto",de vez en cuando,nunca,no ocurrió,no ocurrió,no ocurrió,no ocurrió,no ocurrió,1,0
4,una ciudad,nunca,nunca,no ocurrió,no ocurrió,no ocurrió,no ocurrió,no ocurrió,0,0


In [12]:
# Nos centramos en las col. con la info. que nos interesa para el analisis
analisis_violencia = DF_violencia_mujer[['violencia_infancia','violencia_pareja']]
analisis_violencia.head()

,violencia_infancia,violencia_pareja
0,0,1
1,0,1
2,1,0
3,1,0
4,0,0


In [13]:
# Exportamos el DF final al siguiente notebook 
analisis_violencia.to_csv(
    "../data/processed/analisis_violencia.csv",
    index = False,
    encoding = 'cp1252'
)